In [29]:
import geopandas as gpd

# 행정동 경계 shapefile(.shp) 파일 경로 지정
shp_path = "src/통계청_SGIS 행정구역 통계 및 경계_20240630/2. 경계/3. 2024년 2분기 기준 행정동 경계/bnd_dong_00_2024_2Q.shp"

# GeoDataFrame으로 읽기
dong_gdf = gpd.read_file(shp_path)

# 데이터 확인
print(dong_gdf.head())
# 좌표계 확인
print(dong_gdf.crs)  

  BASE_DATE ADM_NM    ADM_CD  \
0  20240630    충장동  24010510   
1  20240630    동명동  24010540   
2  20240630    안좌면  36680400   
3  20240630    팔금면  36680410   
4  20240630    암태면  36680420   

                                            geometry  
0  POLYGON ((947104.964 1684374.444, 947104.109 1...  
1  POLYGON ((947731.846 1684309.417, 947731.661 1...  
2  MULTIPOLYGON (((867622.6 1643961.356, 867626.7...  
3  MULTIPOLYGON (((878681.738 1650004.079, 878678...  
4  MULTIPOLYGON (((879259.552 1657258.726, 879259...  
EPSG:5179


In [ ]:
import geopandas as gpd

# 1. 데이터 불러오기
dong = gpd.read_file("src/통계청_SGIS 행정구역 통계 및 경계_20240630/2. 경계/3. 2024년 2분기 기준 행정동 경계/bnd_dong_00_2024_2Q.shp") # 행정동 경계
zone = gpd.read_file("src/UQ111_용도지역(도시지역)/shp파일/UPIS_C_UQ111.shp") # 용도지역

# print(dong.head())
# print(zone.head())

# 2. 좌표계 맞추기
zone = zone.to_crs(dong.crs)

# 3. 공간적 교집합 (겹치는 부분만)
overlap = gpd.overlay(zone, dong, how='intersection')

# 4. 겹친 면적 구하기
overlap['area'] = overlap.geometry.area

# 5. 행정동별, 용도지역별로 면적 집계
summary = overlap.groupby(['ADM_NM', 'DGM_NM'])['area'].sum().reset_index()

# 6. 퍼센트로 바꾸려면? (행정동별 전체 면적으로 나누기)
total_area = summary.groupby('ADM_NM')['area'].sum().reset_index().rename(columns={'area':'total_area'})
summary = summary.merge(total_area, on='ADM_NM')
summary['비율'] = summary['area'] / summary['total_area'] * 100

#summary.to_csv("행정동별_용도지역_비율.csv", index=False, encoding='utf-8-sig')

counts = zone['DGM_NM'].value_counts().reset_index()
counts.columns = ['용도지역명', '개수']  # 컬럼명 보기 좋게 변경

def classify_zone(name):
    if '주거' in name:
        return '주거지역'
    elif '상업' in name:
        return '상업지역'
    elif '공업' in name or '준공업' in name or '중공업' in name:
        return '공업지역'
    elif '녹지' in name:
        return '녹지지역'
    else:
        return '기타'

# 대분류 컬럼 추가
counts['대분류'] = counts['용도지역명'].apply(classify_zone)

summary['대분류'] = summary['DGM_NM'].apply(classify_zone)

# 저장
summary.to_csv("행정동별_용도지역_비율2.csv", index=False, encoding="utf-8-sig")

In [40]:
import pandas as pd

# 행정동별 대분류별 비율 데이터 불러오기
grouped = pd.read_csv("행정동별_용도지역_비율2.csv", encoding="utf-8-sig")

# 1. 행정동별 대분류별 비율 합계
zone_sum = grouped.groupby(['ADM_NM', '대분류'])['비율'].sum().reset_index()

# 2. 각 행정동별로 비율이 가장 높은 1, 2순위 대분류 추출
zone_sorted = zone_sum.sort_values(['ADM_NM', '비율'], ascending=[True, False])

# 1순위(최대)와 2순위(두 번째로 큰) 대분류 추출
first = zone_sorted.groupby('ADM_NM').nth(0).reset_index()
second = zone_sorted.groupby('ADM_NM').nth(1).reset_index()

first = first.rename(columns={'대분류': '우세지역1', '비율': '우세지역1_비율'})
second = second.rename(columns={'대분류': '우세지역2', '비율': '우세지역2_비율'})

# 3. 1,2순위 join
result = pd.merge(first, second, on='ADM_NM', how='left')

# 4. 우세지역 판정
def make_major_zone(row):
    if row['우세지역1_비율'] < 50 and pd.notnull(row['우세지역2']):
        return f"{row['우세지역1']}+{row['우세지역2']}"
    else:
        return row['우세지역1']

def make_major_ratio(row):
    if row['우세지역1_비율'] < 50 and pd.notnull(row['우세지역2_비율']):
        return row['우세지역1_비율'] + row['우세지역2_비율']
    else:
        return row['우세지역1_비율']

result['최종우세지역'] = result.apply(make_major_zone, axis=1)
result['최종우세지역_비율'] = result.apply(make_major_ratio, axis=1)

# 5. 최종 컬럼만 선택
final = result[['ADM_NM', '최종우세지역', '최종우세지역_비율']]

# 6. 저장
final.to_csv("행정동별_최종우세지역.csv", index=False, encoding="utf-8-sig")

print(final.head())


  ADM_NM     최종우세지역   최종우세지역_비율
0   가락1동  상업지역+주거지역   73.235908
1   가락2동       주거지역  100.000000
2   가락본동       주거지역   92.314207
3   가리봉동       주거지역   77.681120
4    가산동       공업지역   80.377076


In [1]:
import pandas as pd

df = pd.read_csv('src/RF_parksdf_week.csv')
dong_info = pd.read_csv('행정동별_최종우세지역.csv')

df['동'] = df['address'].str.extract(r'([가-힣0-9]+동)')

우세지역_dict = dong_info.set_index('ADM_NM')['최종우세지역'].to_dict()

df['Majority Area'] = df['동'].map(우세지역_dict)

df.to_csv("RF_parksdf_week_with_area.csv", index=False, encoding="utf-8-sig")

print(df.head())


          address parking_type  operation_type  total_parking_spaces  \
0   강남구 개포동 126-2           NW               1                 132.0   
1  강남구 개포동 1266-0           NW               1                  97.0   
2  강남구 개포동 1273-0           NW               1                 194.0   
3    강남구 개포동 13-2           NW               1                 168.0   
4   강남구 개포동 183-2           NW               1                 149.0   

   base_parking_fee  base_parking_time  additional_unit_fee  \
0             200.0                5.0                200.0   
1             200.0                5.0                200.0   
2             200.0                5.0                200.0   
3             100.0                5.0                100.0   
4             200.0                5.0                200.0   

   additional_unit_time   fee1H  cctv_r300  bus_r300  complaints_r300  \
0                   5.0  2400.0          0         1                1   
1                   5.0  2400.0          0

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# 1. 주차장 데이터: lon, lat → geometry(Point)
df = pd.read_csv('RF_parksdf_week.csv')
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
parking_gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# 2. 용도지역(shp): 폴리곤 데이터 불러오기
zone = gpd.read_file("src/UQ111_용도지역(도시지역)/shp파일/UPIS_C_UQ111.shp")

# 3. 좌표계 일치
if zone.crs != parking_gdf.crs:
    parking_gdf = parking_gdf.to_crs(zone.crs)

# 4. 공간조인: 주차장 포인트가 어느 용도지역 폴리곤에 포함되는지
joined = gpd.sjoin(parking_gdf, zone, how='left', predicate='within')

# 5. 결과 컬럼명 정리 (예: DGM_NM이 용도지역명, 필요시 이름 변경)
joined = joined.rename(columns={'DGM_NM': '용도지역명'})  # 필요시

# 6. 원하는 컬럼만 추려보기
print(joined[['주소', 'lat', 'lon', '용도지역명']].head())

def classify_zone(name):
    if pd.isnull(name):
        return '미분류'    # 또는 '기타', None 등으로 지정
    if '주거' in name: return '주거지역'
    elif '상업' in name: return '상업지역'
    elif '공업' in name or '준공업' in name or '중공업' in name: return '공업지역'
    elif '녹지' in name: return '녹지지역'
    else: return '기타'

joined['용도지역_대분류'] = joined['용도지역명'].apply(classify_zone)

# 7. 파일로 저장 (선택)
joined.to_csv('주차장별_용도지역매칭.csv', index=False, encoding='utf-8-sig')


               주소        lat         lon            용도지역명
0   강남구 개포동 126-2  37.477263  127.066477           자연녹지지역
0   강남구 개포동 126-2  37.477263  127.066477        제3종일반주거지역
1  강남구 개포동 1266-0  37.481496  127.048218        제1종일반주거지역
2  강남구 개포동 1273-0  37.475646  127.052365  제2종일반주거지역(7층이하)
3    강남구 개포동 13-2  37.494938  127.079307        제1종일반주거지역


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from sklearn.preprocessing import LabelEncoder

# 1. 주차장 데이터 불러오기
df = pd.read_csv('src/RF_parksdf_week.csv')

for hour in range(24):
    df[f'is_open_at_{hour}'] = (
        ((df['Weekday_start'] <= hour) & (df['Weekday_end'] > hour)) |
        ((df['Weekday_start'] > df['Weekday_end']) & 
         ((hour >= df['Weekday_start']) | (hour < df['Weekday_end'])))
    ).astype(int)


geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
parking_gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# 2. 용도지역 폴리곤 불러오기
zone = gpd.read_file("src/UQ111_용도지역(도시지역)/shp파일/UPIS_C_UQ111.shp")
if zone.crs != parking_gdf.crs:
    parking_gdf = parking_gdf.to_crs(zone.crs)

# 3. 공간조인
joined = gpd.sjoin(parking_gdf, zone, how='left', predicate='within')
joined = joined.rename(columns={'DGM_NM': 'zone_name'})  # 실제 컬럼명 확인 필요

# 4. 대분류 분류 함수
def classify_zone(name):
    if pd.isnull(name): return '미분류'
    if '주거' in name: return '주거지역'
    elif '상업' in name: return '상업지역'
    elif '공업' in name or '준공업' in name or '중공업' in name: return '공업지역'
    elif '녹지' in name: return '녹지지역'
    else: return '기타'

joined['zoning'] = joined['zone_name'].apply(classify_zone)

# 5. **원래 데이터의 index(혹은 주차장코드)를 사용해 필요한 컬럼만 붙이기**
# - 주차장코드가 유일하다면 그걸로, 아니면 인덱스로

# 예시: 주차장코드가 있다면
cols_to_add = joined[['address', 'zone_name', 'zoning']]
df = df.merge(cols_to_add, on='address', how='left')

le = LabelEncoder()
df['zoning_encoded'] = le.fit_transform(df['zoning'])

# 3. 결과 확인
print(df[['address', 'zoning', 'zoning_encoded']].head())

# 4. 저장
df.to_csv('RF_parksdf_week_with_zone_labeled.csv', index=False, encoding='utf-8-sig')

print(df.head())


          address zoning  zoning_encoded
0   강남구 개포동 126-2   녹지지역               2
1   강남구 개포동 126-2   주거지역               5
2  강남구 개포동 1266-0   주거지역               5
3  강남구 개포동 1273-0   주거지역               5
4    강남구 개포동 13-2   주거지역               5
          address parking_type  operation_type  total_parking_spaces  \
0   강남구 개포동 126-2           NW               1                 132.0   
1   강남구 개포동 126-2           NW               1                 132.0   
2  강남구 개포동 1266-0           NW               1                  97.0   
3  강남구 개포동 1273-0           NW               1                 194.0   
4    강남구 개포동 13-2           NW               1                 168.0   

   base_parking_fee  base_parking_time  additional_unit_fee  \
0             200.0                5.0                200.0   
1             200.0                5.0                200.0   
2             200.0                5.0                200.0   
3             200.0                5.0                200.0   
4     

In [ ]:
import pandas as pd

# 데이터 읽기
df = pd.read_csv('용도지역명_개수_집계.csv', encoding='utf-8-sig')

# 분류 함수 정의
def classify_zone(name):
    if '주거' in name:
        return '주거지역'
    elif '상업' in name:
        return '상업지역'
    elif '공업' in name or '준공업' in name or '중공업' in name:
        return '공업지역'
    elif '녹지' in name:
        return '녹지지역'
    else:
        return '기타'

# 새 컬럼 생성
df['대분류'] = df['용도지역명'].apply(classify_zone)

print(df[['용도지역명', '대분류', '개수']])

df.to_csv('용도지역명_대분류_집계.csv', index=False, encoding='utf-8-sig')

               용도지역명   대분류    개수
0          제2종일반주거지역  주거지역  1871
1          제1종일반주거지역  주거지역  1509
3          제3종일반주거지역  주거지역  1245
4             자연녹지지역  녹지지역   588
5              준주거지역  주거지역   572
6             일반상업지역  상업지역   476
7            기타 도시지역    기타   227
9             근린상업지역  상업지역    54
10             준공업지역  공업지역    49
11         제1종전용주거지역  주거지역    42
14            생산녹지지역  녹지지역    17
15            유통상업지역  상업지역    15
17         제2종전용주거지역  주거지역     4
18            중심상업지역  상업지역     4
19          일반주거지역기타  주거지역     4
20          1종일반주거지역  주거지역     3
22        전용주거지역 미분류  주거지역     3
23          2종일반주거지역  주거지역     2
26          토지구획정리사업    기타     2
29              녹지지역  녹지지역     1
31           재정비촉진구역    기타     1
34  녹번제1구역 제2종일반주거지역  주거지역     1
35       제2종일반주거지역7층  주거지역     1
38            자역녹지지역  녹지지역     1
39         제2종7층일반주거  주거지역     1
44             자연녹지지  녹지지역     1
49            일반주거지역  주거지역     1
51             중공업지역  공업지역     1
54            사회복지시설    기타     1
55     녹번제